# Импортируем библиотеки

In [107]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score

# Загружаем данные

In [108]:
train = pd.read_csv('products_sentiment_train.tsv', sep='\t', names=['text', 'class'])
test = pd.read_csv('products_sentiment_test.tsv', sep='\t')

In [109]:
train.head()

text  class
0          2 . take around 10,000 640x480 pictures .      1
1  i downloaded a trial version of computer assoc...      1
2  the wrt54g plus the hga7t is a perfect solutio...      1
3  i dont especially like how music files are uns...      0
4  i was using the cheapie pail ... and it worked...      1

In [110]:
test.head()

Id                                               text
0   0  so , why the small digital elph , rather than ...
1   1  3/4 way through the first disk we played on it...
2   2  better for the zen micro is outlook compatibil...
3   3    6 . play gameboy color games on it with goboy .
4   4  likewise , i 've heard norton 2004 professiona...

In [111]:
X_train = train['text'].tolist()
y_train = train['class'].tolist()

X_test = test['text'].tolist()

# Проверяем какой pipeline дает наилучшее качество

In [113]:
def text_classifier(vectorizer, classifier):
    return Pipeline([("vectorizer", vectorizer), ("classifier", classifier)])

for vectorizer in [CountVectorizer, TfidfVectorizer]:
    for classifier in [LogisticRegression, LinearSVC, SGDClassifier]:
        print(vectorizer, classifier)
        print(cross_val_score(text_classifier(vectorizer(), classifier()), X_train, y_train).mean())
        print("\n")

<class 'sklearn.feature_extraction.text.CountVectorizer'> <class 'sklearn.linear_model._logistic.LogisticRegression'>
0.7685


<class 'sklearn.feature_extraction.text.CountVectorizer'> <class 'sklearn.svm._classes.LinearSVC'>
0.754


<class 'sklearn.feature_extraction.text.CountVectorizer'> <class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
0.7460000000000001


<class 'sklearn.feature_extraction.text.TfidfVectorizer'> <class 'sklearn.linear_model._logistic.LogisticRegression'>
0.766


<class 'sklearn.feature_extraction.text.TfidfVectorizer'> <class 'sklearn.svm._classes.LinearSVC'>
0.7689999999999999


<class 'sklearn.feature_extraction.text.TfidfVectorizer'> <class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>
0.7545




*Комментарий: наилучшее качество (0.769) дает TfidfVectorizer + LinearSVC*

# Перебираем параметры, которые позволяют улучшить качество классификатора TfidfVectorizer + LinearSVC

**Смотрим при каких ngram_range, analyzer, stop_words получается лучшее качество**

In [150]:
import nltk
from nltk.corpus import stopwords

In [148]:
params = []
clfs_quality_measure = []

for analyzer in ['word', 'char_wb']:
    for stop_words in [stopwords.words('english'), 'english', None]:
        for ngram_range_lower in [1, 2, 3, 4, 5, 6]:
            for ngram_range_upper in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]:
                if ngram_range_upper >= ngram_range_lower:
                    if analyzer == 'char_wb' and stop_words == None or analyzer == 'word':
                        clf_quality = cross_val_score(text_classifier(TfidfVectorizer(ngram_range=(ngram_range_lower, ngram_range_upper), analyzer=analyzer, stop_words=stop_words), LinearSVC()), X_train, y_train).mean()            
#                         print("Ngram range: {} - {}".format(ngram_range_lower, ngram_range_upper))
#                         print("Analyzer: {}".format(analyzer))
#                         print("Stop words: {}".format(stop_words))                        
#                         print(clf_quality)
#                         print("\n")
                        params.append({'ngram lower range': ngram_range_lower, 'ngram upper range': ngram_range_upper, 'analyzer': analyzer, 'stop words': stop_words})
                        clfs_quality_measure.append(clf_quality)
        
best_quality = max(clfs_quality_measure)
best_quality_idx = clfs_quality_measure.index(best_quality)
best_params = params[best_quality_idx]
print("Best classificator quality:", best_quality)
print("Index of best classificator:", best_quality_idx)
print("Best params:", best_params)

Best classificator quality: 0.79
Index of best classificator: 116
Best params: {'ngram lower range': 1, 'ngram upper range': 3, 'analyzer': 'word', 'stop words': None}


*Комментарий: лучшее качество (0.79) получается при ngram_range=(1,3), остальные параметры по умалчанию*

**Смотрим при каких min_df и max_df получается лучшее качество**

In [149]:
params = []
clfs_quality_measure = []

for max_df in [1.0, 0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.9, 0.89, 0.88, 0.87, 0.86, 0.85]:
    for min_df in [1, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]:
        clf_quality = cross_val_score(text_classifier(TfidfVectorizer(ngram_range=(1,3), analyzer='word', stop_words=None, max_df=max_df, min_df=min_df), LinearSVC()), X_train, y_train).mean()            
#         print("max_df: {}".format(max_df))
#         print("min_df: {}".format(min_df))                        
#         print(clf_quality)
#         print("\n")
        params.append({'max_df': max_df, 'min_df': min_df})
        clfs_quality_measure.append(clf_quality)
        
best_quality = max(clfs_quality_measure)
best_quality_idx = clfs_quality_measure.index(best_quality)
best_params = params[best_quality_idx]
print("Best classificator quality:", best_quality)
print("Index of best classificator:", best_quality_idx)
print("Best params:", best_params)

Best classificator quality: 0.79
Index of best classificator: 0
Best params: {'max_df': 1.0, 'min_df': 1}


*Комментарий: лучшее качество (0.79) получается при max_df и min_df по умалчанию*

# Создаем pipeline для классификатора TfidfVectorizer + LinearSVC и делаем предсказания на тесовом датасете

In [154]:
pipe = Pipeline([("vectorizer", TfidfVectorizer(ngram_range=(1,3))), ("classifier", LinearSVC())])
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 3), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
   

In [155]:
predictions = pipe.predict(X_test)
test_index = test['Id'].tolist()

In [156]:
submission_data = {'Id': test_index, 'y': predictions}
submission = pd.DataFrame(submission_data)
submission.to_csv('submission.csv', index=False)